# Data Cleaning and Processing

In [19]:
# takes a minite to run
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
chunks = pd.read_csv("EastCoastDatathonMaterials/nyc_bikeshare.csv", chunksize=10000)
for chunk in chunks:
    NYC_bikeshare = chunk
    break
#NYC_bikeshare = pd.read_csv("EastCoastDatathonMaterials/nyc_bikeshare.csv", chunksize=100)
NYC_bikeshare

,tripduration,starttime,stoptime,start_station_id,end_station_id,bikeid,usertype,birthyear,gender
0,2059,06-01-13 00:00:44,06-01-13 00:35:03,406.0,406.0,19599,0.0,NaN,0
1,1521,06-01-13 00:01:22,06-01-13 00:26:43,2008.0,310.0,15567,1.0,1983.0,1
2,2028,06-01-13 00:01:47,06-01-13 00:35:35,485.0,406.0,18445,0.0,NaN,0
3,1829,06-01-13 00:03:47,06-01-13 00:34:16,265.0,436.0,15234,1.0,1984.0,1
4,899,06-01-13 00:09:25,06-01-13 00:24:24,494.0,494.0,15539,1.0,1967.0,1
...,...,...,...,...,...,...,...,...,...
9995,690,06-03-13 23:54:35,06-04-13 00:06:05,312.0,461.0,16843,0.0,NaN,0
9996,783,06-03-13 23:55:19,06-04-13 00:08:22,416.0,368.0,19152,1.0,1969.0,1
9997,1029,06-03-13 23:56:38,06-04-13 00:13:47,416.0,455.0,19776,0.0,NaN,0
9998,1076,06-03-13 23:58:33,06-04-13 00:16:29,432.0,2023.0,15004,0.0,NaN,0


In [20]:
NYC_bikeshare['starttime'] = pd.to_datetime(NYC_bikeshare['starttime'])
NYC_bikeshare['stoptime'] = pd.to_datetime(NYC_bikeshare['stoptime'])


In [39]:
NYC_weather = pd.read_csv("EastCoastDatathonMaterials/weather.csv")
NYC_weather['DATE'] = pd.to_datetime(NYC_weather['DATE'])

/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
NYC_weather_hourly = NYC_weather[NYC_weather['REPORT_TYPE'] == 'FM-15']
NYC_weather_hourly = NYC_weather_hourly[['DATE', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyWindSpeed']]
NYC_weather_hourly.columns = ['Date', 'DewTemp', 'DryTemp', 'Precip', 'Wspeed']
NYC_bikeshare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
tripduration        10000 non-null int64
starttime           10000 non-null datetime64[ns]
stoptime            10000 non-null datetime64[ns]
start_station_id    10000 non-null float64
end_station_id      9657 non-null float64
bikeid              10000 non-null int64
usertype            10000 non-null float64
birthyear           4642 non-null float64
gender              10000 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(3)
memory usage: 703.2 KB


In [41]:
NYC_weather_hourly

,Date,DewTemp,DryTemp,Precip,Wspeed
0,2013-01-01 00:51:00,24.0,39,0.0,10.0
1,2013-01-01 01:51:00,26.0,38,0.0,3.0
2,2013-01-01 02:51:00,27.0,39,0.0,7.0
3,2013-01-01 03:51:00,27.0,39,0.0,9.0
4,2013-01-01 04:51:00,27.0,39,0.0,7.0
...,...,...,...,...,...
81389,2019-12-31 11:51:00,35,40,NaN,5.0
81390,2019-12-31 12:51:00,36,41,NaN,NaN
81391,2019-12-31 13:51:00,37,43,NaN,6.0
81392,2019-12-31 14:51:00,36,43,NaN,7.0


In [47]:
def myfunc(date):
    return date.replace(microsecond=0,second=0,minute=0)
NYC_weather_hourly['Date'] = NYC_weather_hourly['Date'].apply(myfunc)

def checkweekday(date):
    return date.weekday()
NYC_weather_hourly['weekday'] = NYC_weather_hourly['Date'].apply(checkweekday)

In [48]:
cal = calendar()
holidays = cal.holidays(start=NYC_weather_hourly['Date'].min(), end=NYC_weather_hourly['Date'].max())

def checkholiday(date):
    return date.replace(hour=0) in holidays
NYC_weather_hourly['holiday'] = NYC_weather_hourly['Date'].apply(checkholiday)
# TODO: Add snow feature (0, 1) from daily data

In [49]:
NYC_weather_hourly

,Date,DewTemp,DryTemp,Precip,Wspeed,weekday,holiday
0,2013-01-01 00:00:00,24.0,39,0.0,10.0,1,True
1,2013-01-01 01:00:00,26.0,38,0.0,3.0,1,True
2,2013-01-01 02:00:00,27.0,39,0.0,7.0,1,True
3,2013-01-01 03:00:00,27.0,39,0.0,9.0,1,True
4,2013-01-01 04:00:00,27.0,39,0.0,7.0,1,True
...,...,...,...,...,...,...,...
81389,2019-12-31 11:00:00,35,40,NaN,5.0,1,False
81390,2019-12-31 12:00:00,36,41,NaN,NaN,1,False
81391,2019-12-31 13:00:00,37,43,NaN,6.0,1,False
81392,2019-12-31 14:00:00,36,43,NaN,7.0,1,False


In [51]:
NYC_bikeshare['DDay'] = NYC_bikeshare['starttime'].apply(myfunc)


In [52]:
num_trips = NYC_bikeshare.groupby(['DDay', 'usertype']).count()[['tripduration']]
num_trips.reset_index(inplace=True)
num_trips = num_trips.pivot(index='DDay', columns='usertype', values='tripduration')
num_trips.columns=['cust_cnt', 'subs_cnt']

trip_duration = NYC_bikeshare.groupby(['DDay', 'usertype']).sum()[['tripduration']]
trip_duration.reset_index(inplace=True)
trip_duration = trip_duration.pivot(index='DDay', columns='usertype', values='tripduration')
trip_duration.columns=['cust_dur', 'subs_dur']

num_trips

,cust_cnt,subs_cnt
DDay,,
2013-06-01 00:00:00,5.0,48.0
2013-06-01 01:00:00,7.0,22.0
2013-06-01 02:00:00,6.0,9.0
2013-06-01 03:00:00,8.0,8.0
2013-06-01 04:00:00,NaN,4.0
...,...,...
2013-06-03 19:00:00,101.0,159.0
2013-06-03 20:00:00,71.0,134.0
2013-06-03 21:00:00,42.0,76.0


In [53]:
trip_duration

,cust_dur,subs_dur
DDay,,
2013-06-01 00:00:00,10649.0,339932.0
2013-06-01 01:00:00,8628.0,23318.0
2013-06-01 02:00:00,8574.0,66994.0
2013-06-01 03:00:00,20410.0,6796.0
2013-06-01 04:00:00,NaN,3017.0
...,...,...
2013-06-03 19:00:00,133611.0,333973.0
2013-06-03 20:00:00,137701.0,461903.0
2013-06-03 21:00:00,83511.0,265417.0


In [54]:
# merge into NYC_weather_hourly
A = NYC_weather_hourly.set_index('Date')
mergeddf = A.merge(num_trips, left_index=True, right_index=True)
mergeddf

,DewTemp,DryTemp,Precip,Wspeed,weekday,holiday,cust_cnt,subs_cnt
2013-06-01 00:00:00,64.0,77,0.0,5.0,5,False,5.0,48.0
2013-06-01 01:00:00,65.0,76,0.0,NaN,5,False,7.0,22.0
2013-06-01 02:00:00,65.0,76,0.0,5.0,5,False,6.0,9.0
2013-06-01 03:00:00,65.0,74,0.0,0.0,5,False,8.0,8.0
2013-06-01 04:00:00,65.0,74,0.0,6.0,5,False,NaN,4.0
...,...,...,...,...,...,...,...,...
2013-06-03 19:00:00,68.0,70,0.0,0.0,0,False,101.0,159.0
2013-06-03 20:00:00,59.0,69,0.0,5.0,0,False,71.0,134.0
2013-06-03 21:00:00,54.0,68,0.0,3.0,0,False,42.0,76.0
2013-06-03 22:00:00,46.0,68,0.0,3.0,0,False,57.0,54.0


In [ ]:
#####################
# Useless code
#####################
# construct idea 1 dataframe
# start_time = max(np.min(NYC_bikeshare['starttime']),np.min(NYC_weather['DATE'])).replace(microsecond=0,second=0,minute=0)
# end_time = min(np.max(NYC_bikeshare['starttime']), np.max(NYC_weather['DATE'])).replace(microsecond=0,second=0,minute=0) + timedelta(hours=1)
# for testing:
start_time = min(np.min(NYC_bikeshare['starttime']),np.min(NYC_weather['DATE'])).replace(microsecond=0,second=0,minute=0)
end_time = max(np.max(NYC_bikeshare['starttime']), np.max(NYC_weather['DATE'])).replace(microsecond=0,second=0,minute=0) + timedelta(hours=1)
a = pd.date_range(start=start_time, end=end_time, freq='H')